In [1]:
import sys
import os
import json
import random
import re
import socket
import threading
import time



HOST = '127.0.0.1'  # localhost
PORT = 5500



# Jupyter 환경 대응: 현재 작업 디렉토리를 기준으로 상위 폴더의 data 폴더 경로 설정
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
data_dir = os.path.join(parent_dir, 'data')

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

def make_emotion_json(filename):
    emotions = {
        "joy": round(random.uniform(0, 1), 3),
        "sadness": round(random.uniform(0, 1), 3),
        "surprise": round(random.uniform(0, 1), 3),
        "anger": round(random.uniform(0, 1), 3),
        "calm": round(random.uniform(0, 1), 3)
    }

    # 우세 감정 결정
    dominant_emotion = max(emotions, key=emotions.get)

    data = emotions.copy()
    data["dominant_emotion"] = dominant_emotion

    filepath = os.path.join(data_dir, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"{filepath} 파일이 생성되었습니다.")




def get_next_index():
    files = [f for f in os.listdir(data_dir) if re.match(r'Emotion_Metadata\(\d+\)\.json', f)]
    if not files:
        return 0
    indices = [int(re.findall(r'Emotion_Metadata\((\d+)\)\.json', f)[0]) for f in files]
    return max(indices) + 1



def delete_emotion_files():
    files = [f for f in os.listdir(data_dir) if f.startswith('Emotion_Metadata') and (f.endswith('.json') or f.endswith('.meta'))]
    for f in files:
        os.remove(os.path.join(data_dir, f))
        print(f"{f} 파일 삭제됨.")
    print("모든 Emotion_Metadata json/meta 파일 삭제 완료.")



class Server:
    def __init__(self):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.bind((HOST, PORT))
        self.sock.listen()
        self.conn = None
        self.addr = None
        self.commencing = False
        self.commencement_time = None
        self.send_thread = None
        self.send_flag = threading.Event()
        self.timeout_thread = None



    def accept_client_with_timeout(self, timeout=60):
        self.sock.settimeout(timeout)
        try:
            print(f"서버가 {HOST}:{PORT} 에서 대기 중입니다. 최대 {timeout}초 대기...")
            self.conn, self.addr = self.sock.accept()
            print(f'Connected by {self.addr}')
            self.sock.settimeout(None)  # Timeout 해제
            return True
        except socket.timeout:
            print(f"{timeout}초 동안 연결을 기다렸으나 클라이언트가 연결되지 않았습니다.")
            return False



    def wait_for_connection_timeout_thread(self, timeout=60):
        start_time = time.time()
        while self.commencing and (time.time() - start_time) < timeout:
            if self.conn is not None:
                return  # 연결 됨
            time.sleep(0.5)
        if self.conn is None:
            print("클라이언트 연결 타임아웃으로 자동 통신 종료합니다.")
            self.stop_communication()



    def send_new_files(self):
        last_checked = self.commencement_time
        while self.send_flag.is_set():
            files = [f for f in os.listdir(data_dir) if re.match(r'Emotion_Metadata\(\d+\)\.json', f)]
            new_files = []
            for f in files:
                filepath = os.path.join(data_dir, f)
                if os.path.getmtime(filepath) > last_checked:
                    new_files.append(f)
            new_files.sort(key=lambda x: int(re.findall(r'Emotion_Metadata\((\d+)\)\.json', x)[0]))



            for nf in new_files:
                filepath = os.path.join(data_dir, nf)
                abs_path = os.path.abspath(filepath)  # 절대 경로 얻기
                timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(os.path.getmtime(filepath)))
                
                send_data = json.dumps({
                    "filename": nf,
                    "time": timestamp,
                    "fullpath": abs_path
                })

                try:
                    self.conn.sendall(send_data.encode('utf-8'))
                    print(f"데이터 전송됨: {send_data}")
                except Exception as e:
                    print(f"전송 오류: {e}")
                    self.send_flag.clear()
                    return
                time.sleep(1)
            last_checked = time.time()
            time.sleep(2)



    def start_communication(self):
        if self.commencing:
            print("이미 통신이 시작된 상태입니다.")
            return
        self.commencing = True
        self.commencement_time = time.time()
        
        # 클라이언트 연결 대기 (타임아웃 60초)
        connected = self.accept_client_with_timeout(timeout=60)
        if not connected:
            self.stop_communication()
            return
        
        self.send_flag.set()
        self.send_thread = threading.Thread(target=self.send_new_files)
        self.send_thread.start()



        # 타임아웃 감시 스레드 시작 (연결 확인 용, 필요에 따라 제거 가능)
        self.timeout_thread = threading.Thread(target=self.wait_for_connection_timeout_thread, args=(60,))
        self.timeout_thread.start()



        print("통신이 시작되었습니다.")



    def stop_communication(self):
        if not self.commencing:
            print("통신이 시작된 상태가 아닙니다.")
            return
        self.commencing = False
        self.send_flag.clear()
        if self.send_thread:
            self.send_thread.join()
        if self.timeout_thread:
            self.timeout_thread.join()
        if self.conn:
            self.conn.close()
            self.conn = None
        print("통신이 종료되었습니다.")



def user_input_thread(server):
    try:
        while True:
            user_input = input("명령어 입력: c(통신 시작/종료), l(랜덤 감정 데이터 생성), d(감정 파일 전체 삭제), p(프로그램 종료): ")
            if user_input.lower() == 'c':
                if server.commencing:
                    server.stop_communication()
                else:
                    try:
                        server.start_communication()
                    except Exception as e:
                        print(f"클라이언트 연결 오류: {e}")
            elif user_input.lower() == 'l':
                idx = get_next_index()
                filename = f"Emotion_Metadata({idx}).json"
                make_emotion_json(filename)
            elif user_input.lower() == 'd':
                delete_emotion_files()
            elif user_input.lower() == 'p':
                if server.commencing:
                    server.stop_communication()
                print("프로그램을 종료합니다.")
                sys.exit()
    except KeyboardInterrupt:
        if server.commencing:
            server.stop_communication()
        sys.exit()



def main():
    server = Server()
    input_thread = threading.Thread(target=user_input_thread, args=(server,), daemon=True)
    input_thread.start()
    input_thread.join()



if __name__ == "__main__":
    main()


서버가 127.0.0.1:5500 에서 대기 중입니다. 최대 60초 대기...
Connected by ('127.0.0.1', 50627)
통신이 시작되었습니다.
c:\WorkSpace\Hub_WorkSpace\Github_WorkSpace\Saek-Index_Emotion-Visualization\Saek-Index\Assets\Alpha_Dev\Script\data\Emotion_Metadata(0).json 파일이 생성되었습니다.
데이터 전송됨: {"filename": "Emotion_Metadata(0).json", "time": "2025-09-14 19:48:20", "fullpath": "c:\\WorkSpace\\Hub_WorkSpace\\Github_WorkSpace\\Saek-Index_Emotion-Visualization\\Saek-Index\\Assets\\Alpha_Dev\\Script\\data\\Emotion_Metadata(0).json"}
데이터 전송됨: {"filename": "Emotion_Metadata(0).json.meta", "time": "2025-09-14 19:48:21", "fullpath": "c:\\WorkSpace\\Hub_WorkSpace\\Github_WorkSpace\\Saek-Index_Emotion-Visualization\\Saek-Index\\Assets\\Alpha_Dev\\Script\\data\\Emotion_Metadata(0).json.meta"}
c:\WorkSpace\Hub_WorkSpace\Github_WorkSpace\Saek-Index_Emotion-Visualization\Saek-Index\Assets\Alpha_Dev\Script\data\Emotion_Metadata(1).json 파일이 생성되었습니다.
데이터 전송됨: {"filename": "Emotion_Metadata(1).json", "time": "2025-09-14 19:48:35", "fullpath": 